# Installation

In [1]:
#!pip install folium
#!pip install altair

# Imports

In [1]:
import pandas as pd
import folium
from folium.features import FeatureGroup, VegaLite
from folium.plugins import MarkerCluster
import altair as alt

# Process Data

In [2]:
# Read 'xlsx' file as pandas dataframe
df = pd.read_excel(r'data/hospitales.xlsx', converters={'Long':str, 'Lat':str}, index_col=None)
# Remove rows based on a column value
df = df.drop(df['Long'].loc[df['Long']=="0"].index)

# Reset index
df.reset_index(drop=True, inplace=True)
# Create new column with index values
df.insert(loc=0, column='Id', value=df.index)

# Create a copy of dataframe and delete columns
df_copy = df.drop(columns=['Long', 'Lat', 'Total', 'Hospital'])

# Add a label to a column
df_copy['Id'] = 'Hospital' + df['Id'].astype(str)

df_copy = df_copy.set_index('Id')

# Transpose dataframe
df_tr = df_copy.transpose()

tipos = ['Maltrato1', 'Maltrato2', 'Maltrato3', 'Maltrato4', 'Maltrato5', 
         'Maltrato6', 'Maltrato7', 'Maltrato8', 'Maltrato9', 'Maltrato10']

maltratos = ['Te llamaron por sobrenombres o diminutivos (mamita, niña, gorda, negra, etc)', 
             'Se burlaron de ti', 
             'Te dijeron que no explícitamente a algo que pediste', 
             'Te hablaron en voz alta de modo grosero', 
             'Ignoraron algo que les pediste', 
             'Te dieron golpes, sacudidas, empujones', 
             'Te dijeron que estabas poniendo en riesgo a tu hijo/a', 
             'Te culparon por situaciones ocurridas en el nacimiento', 
             'Te amenazaron', 
             'Otro tipo de maltrato']

df_tr.insert(loc=0, column='Tipos', value=tipos)
df_tr.insert(loc=1, column='Maltratos', value=maltratos)

df_tr

Id,Tipos,Maltratos,Hospital0,Hospital1,Hospital2,Hospital3,Hospital4,Hospital5,Hospital6,Hospital7,...,Hospital9,Hospital10,Hospital11,Hospital12,Hospital13,Hospital14,Hospital15,Hospital16,Hospital17,Hospital18
Maltrato1,Maltrato1,Te llamaron por sobrenombres o diminutivos (ma...,1.0,1.0,2.0,19.0,3.0,3.0,1.0,1.0,...,10.0,1.0,2.0,2.0,3.0,3.0,6.0,1.0,2.0,14.0
Maltrato2,Maltrato2,Se burlaron de ti,NaN,NaN,1.0,8.0,2.0,1.0,NaN,1.0,...,4.0,1.0,2.0,1.0,1.0,1.0,3.0,NaN,1.0,4.0
Maltrato3,Maltrato3,Te dijeron que no explícitamente a algo que pe...,NaN,1.0,1.0,22.0,3.0,5.0,NaN,2.0,...,9.0,2.0,2.0,1.0,4.0,1.0,6.0,NaN,1.0,17.0
Maltrato4,Maltrato4,Te hablaron en voz alta de modo grosero,NaN,1.0,1.0,27.0,4.0,6.0,1.0,1.0,...,10.0,2.0,2.0,2.0,5.0,1.0,9.0,1.0,1.0,27.0
Maltrato5,Maltrato5,Ignoraron algo que les pediste,NaN,1.0,1.0,18.0,4.0,5.0,NaN,1.0,...,9.0,2.0,2.0,2.0,4.0,NaN,7.0,1.0,1.0,16.0
Maltrato6,Maltrato6,"Te dieron golpes, sacudidas, empujones",NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,1.0,2.0,1.0,1.0,NaN,1.0,NaN,NaN,1.0
Maltrato7,Maltrato7,Te dijeron que estabas poniendo en riesgo a tu...,NaN,NaN,1.0,16.0,3.0,1.0,NaN,1.0,...,3.0,2.0,2.0,2.0,2.0,NaN,3.0,2.0,1.0,12.0
Maltrato8,Maltrato8,Te culparon por situaciones ocurridas en el na...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,...,2.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,3.0
Maltrato9,Maltrato9,Te amenazaron,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,1.0,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,3.0
Maltrato10,Maltrato10,Otro tipo de maltrato,1.0,1.0,NaN,17.0,1.0,NaN,1.0,1.0,...,2.0,1.0,1.0,2.0,5.0,NaN,4.0,NaN,2.0,18.0


# Example with Circle Markers and Popups with Bar Charts

In [3]:
# Plot charts
chart = alt.Chart(df_tr).mark_bar(color='blue', size=20).encode(
        alt.X("Hospital5:Q", title='Hospital', scale=alt.Scale(domain=[0, 30])),
        alt.Y("Tipos:N", title='Tipos', sort=alt.EncodingSortField(field="Hospital5", op="count", order='ascending')),
        tooltip="Maltratos").interactive().properties(title='Tipos de maltrato por Hospital', width=350, height=300)

# Configure chart title
chart = chart.configure_title(
        fontSize=15,
        font='Courier',
        anchor='middle',
        color='black')

# Convert chart to json
vis = chart.to_json()


# Create a map
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, tiles=None)

# Add layers
# Dark Mode
folium.TileLayer(tiles="CartoDB dark_matter", name="Dark", attr= '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>').add_to(m)
# Light Mode
folium.TileLayer(tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png', name="Light", attr= '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>').add_to(m)

# Add the option to switch layers
hospitales = FeatureGroup(name="Hospitales")
hospitales.add_to(m)
folium.LayerControl().add_to(m)

# Add circle markers
for i, r in df.iterrows():
    folium.CircleMarker([r['Long'], r['Lat']],
    radius=r['Total']/3,
    color='#9b00ff',
    fill_color='#9b00ff',
    fill_opacity=(0.5),
    tooltip=r['Hospital'],
    # Add popup with chart                    
    popup=folium.Popup(max_width=450, max_heigth=400).add_child(folium.VegaLite(vis, width=450, height=300)),
    icon=folium.Icon(icon_color='white', color='red')).add_to(hospitales)

m

In [234]:
# Save dataframe as 'xlsx'
#df_tr.to_excel(f'data/maltratos_hospital.xlsx', index = False)